# The code here is for the analysis of membership type 
- Comparing subscriber and customer in total trip count and trip count per city
- (Not This Part) Comparing subscriber and customer bike share usage on different weather at different time of the year

### ------- Reading data and creating columns --------

In [ ]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format   # all results in 2 decimal points
#from ggplot import *

In [ ]:
# formatting lambdas & functions
#fnone = lambda x : '{0:g}'.format(float(x))
#fshort = lambda x : '{:,.2f}'.format(x)

def convert_float(val):
    try:
        return float(val)
    except ValueError:
        return 0

# dictionary reference
seasons_dict = {
    1: 'Winter',2: 'Spring',3: 'Spring',4: 'Spring',
    5: 'Summer',6: 'Summer',7: 'Summer',8: 'Autumn',
    9: 'Autumn',10: 'Autumn',11: 'Winter',12: 'Winter'}

city_dict = {
    94107: 'San Francisco',
    94063: 'Redwood City',
    94301: 'Palo Alto', 
    94041: 'Mountain View', 
    95113: 'San Jose'}

# header reference
station_headers = ['station_id','station_name','lat','long',
                   'dock_count','city','install_date']

status_headers = ['station_id','bikes_free','docks_free','time']

trip_headers = ['trip_id','trip_time',
                'start_dt','start_station','start_terminal',
                'end_dt','end_station','end_terminal',
                'bike_num','user_type','ZIP']

weather_headers = ['date',
                   'max_temp','mean_temp','min_temp',
                   'max_dp','mean_dp','min_dp',
                   'max_hum','mean_hum','min_hum',
                   'max_sea','mean_sea','min_sea',
                   'max_vis','mean_vis','min_vis',
                   'max_wind','mean_wind','max_gust',
                   'rain_inches','cloud_cover','events',
                   'wind_dir_degrees','ZIP']

In [ ]:
# read trip data for 2014 
raw_data_201402 = pd.read_csv('201402_trip_data_new.csv', parse_dates=True)
raw_data_201402.columns = trip_headers
raw_data_201408 = pd.read_csv('201408_trip_data_new.csv', parse_dates=True)
raw_data_201408.columns = trip_headers

In [ ]:
# joining 201402 and 201408 together
raw_data_2014 = pd.concat([raw_data_201402, raw_data_201408])
# convert duration column "trip_time" to unit of minutes
raw_data_2014['trip_time'] = raw_data_2014['trip_time'] / 60   # trip time unit in minute

In [ ]:
# read trip data for 2015
raw_data_2015 = pd.read_csv('201508_trip_data_new.csv', parse_dates=True)
raw_data_2015.columns = trip_headers
raw_data_2015['trip_time'] = raw_data_2015['trip_time'] / 60 # trip time unit in minute 

## Part 1 - Membership count in total and per city

### ------------- Year 2014 ------------- 

In [ ]:
raw_data_2014.head()

In [ ]:
# create new dataframes with columns related to only trip time (duration) and user type (subscription)
duration_data_2014 = pd.DataFrame(raw_data_2014, columns=['user_type', 'trip_time'])
duration_data_2014.columns = ['membership', 'trip_duration']  
duration_data_2014.head()

## Total Count and membership in total

In [ ]:
# duration description in total
duration_data_2014.describe()

In [ ]:
# membership count (same as trip count) counts in total
membership_2014 = duration_data_2014.groupby('membership')
count_2014 = membership_2014.count()['trip_duration']
count_2014.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Membership Total Count in Percentage 2014', fontsize=20)

In [ ]:
# statistics table for total customer and subscribers in 2014
membership_2014.describe()

In [ ]:
# make a new table with min, max, mean for both membership
mean_2014 = membership_2014.mean()['trip_duration'] / 60  # unit in hour
min_2014 = membership_2014.min()['trip_duration'] / 60 # unit in hour  
mid_2014 = membership_2014.quantile(0.5)['trip_duration'] / 60  # unit in hour
#max_2014 = membership_2014.max()['trip_duration'] / 60 # unit in hour ---> too big

member_time_2014 = pd.DataFrame({'Average': mean_2014, '50 percentile': mid_2014, 'Minimum': min_2014})
member_time_2014.plot.bar(figsize=(8,8),fontsize=15)
plt.xlabel('User Duration of Trip Total Count 2014', fontsize=25)

## Trip Count and membershio info per city (SF, RC, PA, SJ, MV)

In [ ]:
# STATION_DATA
# read station data, grou
raw_data_1 = pd.read_csv('201402_station_data.csv')
raw_data_1.columns = station_headers
raw_data_2 = pd.read_csv('201408_station_data.csv')
raw_data_2.columns = station_headers
raw_data_3 = pd.read_csv('201508_station_data.csv') # current dataset
raw_data_3.columns = station_headers

# merged station datasets
station_data = pd.concat([raw_data_1, raw_data_2, raw_data_3])
station_data.head()

In [ ]:
# sort station ID by city
san_jose_station_id = list(set(station_data[station_data['city']=='San Jose']['station_id']))
san_francisco_station_id = list(set(station_data[station_data['city']=='San Francisco']['station_id']))
redwood_city_station_id = list(set(station_data[station_data['city']=='Redwood City']['station_id']))
palo_alto_station_id = list(set(station_data[station_data['city']=='Palo Alto']['station_id']))
mountain_view_station_id = list(set(station_data[station_data['city']=='Mountain View']['station_id']))

In [ ]:
# filter raw trip data 2014 according to station ID
sf_trip_data_2014 = raw_data_2014[raw_data_2014['start_terminal'].isin(palo_alto_station_id)]
sj_trip_data_2014 = raw_data_2014[raw_data_2014['start_terminal'].isin(san_jose_station_id)]
rc_trip_data_2014 = raw_data_2014[raw_data_2014['start_terminal'].isin(redwood_city_station_id)]
pa_trip_data_2014 = raw_data_2014[raw_data_2014['start_terminal'].isin(palo_alto_station_id)]
mv_trip_data_2014 = raw_data_2014[raw_data_2014['start_terminal'].isin(mountain_view_station_id)]

### San Francisco Trip Count and Membership info

In [ ]:
sf_trip_data_2014 = pd.DataFrame(sf_trip_data_2014, columns=['user_type', 'trip_time'])
sf_trip_data_2014.columns = ['membership', 'trip_duration']  
sf_trip_data_2014.head()

In [ ]:
# SF trip data statistics for both user type
sf_trip_data_2014.describe()

In [ ]:
# SF trip data for each user type
sf_trip_data_membership_2014.describe()

In [ ]:
# membership count (same as trip count) counts in San Francisco
sf_trip_data_membership_2014 = sf_trip_data_2014.groupby('membership')
sf_count_2014 = sf_trip_data_membership_2014.count()['trip_duration']
sf_count_2014.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in San Francisco in Percentage 2014', fontsize=20)

### San Jose Trip Count and Membership info

In [ ]:
sj_trip_data_2014 = pd.DataFrame(sj_trip_data_2014, columns=['user_type', 'trip_time'])
sj_trip_data_2014.columns = ['membership', 'trip_duration']  
sj_trip_data_2014.head()

In [ ]:
# SJ trip data statistics for both user type
sj_trip_data_2014.describe()

In [ ]:
# membership count (same as trip count) counts in San Jose
sj_trip_data_membership_2014 = sj_trip_data_2014.groupby('membership')
sj_count_2014 = sj_trip_data_membership_2014.count()['trip_duration']
sj_count_2014.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in San Jose in Percentage 2014', fontsize=20)

In [ ]:
# SJ trip data for each user type
sj_trip_data_membership_2014.describe()

### Palo Alto Trip Count and Membership info

In [ ]:
pa_trip_data_2014 = pd.DataFrame(pa_trip_data_2014, columns=['user_type', 'trip_time'])
pa_trip_data_2014.columns = ['membership', 'trip_duration']  
pa_trip_data_2014.head()

In [ ]:
# PA trip data statistics for both user type
pa_trip_data_2014.describe()

In [ ]:
# membership count (same as trip count) counts in Palo Alto
pa_trip_data_membership_2014 = pa_trip_data_2014.groupby('membership')
pa_count_2014 = pa_trip_data_membership_2014.count()['trip_duration']
pa_count_2014.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in Palo Alto in Percentage 2014', fontsize=20)

In [ ]:
# PA trip data for each user type
pa_trip_data_membership_2014.describe()

### Redwood CIty Trip Count and Membership info

In [ ]:
rc_trip_data_2014 = pd.DataFrame(rc_trip_data_2014, columns=['user_type', 'trip_time'])
rc_trip_data_2014.columns = ['membership', 'trip_duration']  
rc_trip_data_2014.head()

In [ ]:
# RC trip data statistics for both user type
rc_trip_data_2014.describe()

In [ ]:
# membership count (same as trip count) counts in Redwood City
rc_trip_data_membership_2014 = rc_trip_data_2014.groupby('membership')
rc_count_2014 = rc_trip_data_membership_2014.count()['trip_duration']
rc_count_2014.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in Redwood City in Percentage 2014', fontsize=20)

In [ ]:
# RC trip data for each user type
rc_trip_data_membership_2014.describe()

### Mountain View Trip Count and Membership info

In [ ]:
mv_trip_data_2014 = pd.DataFrame(mv_trip_data_2014, columns=['user_type', 'trip_time'])
mv_trip_data_2014.columns = ['membership', 'trip_duration']  
mv_trip_data_2014.head()

In [ ]:
# MV trip data statistics for both user type
mv_trip_data_2014.describe()

In [ ]:
# membership count (same as trip count) counts in Mountain View
mv_trip_data_membership_2014 = mv_trip_data_2014.groupby('membership')
mv_count_2014 = mv_trip_data_membership_2014.count()['trip_duration']
mv_count_2014.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in Mountain View in Percentage 2014', fontsize=20)

In [ ]:
# MV trip data for each user type
mv_trip_data_membership_2014.describe()

### ------------- Year 2015 ------------- 

In [ ]:
raw_data_2015.head()

In [ ]:
# create new dataframes with columns related to only trip time (duration) and user type (subscription)
duration_data_2015 = pd.DataFrame(raw_data_2015, columns=['user_type', 'trip_time'])
duration_data_2015.columns = ['membership', 'trip_duration']  
duration_data_2015.head()

## Total Count and member in total

In [ ]:
# duration description in total --> notice count increases 
duration_data_2015.describe()

In [ ]:
# membership count (same as trip count) counts in total
membership_2015 = duration_data_2015.groupby('membership')
count_2015 = membership_2015.count()['trip_duration']
count_2015.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Membership Total Count in Percentage 2015', fontsize=20)

In [ ]:
# statistics table for total customer and subscribers in 2015
membership_2015.describe()

In [ ]:
# make a new table with min, max, mean for both membership
mean_2015 = membership_2015.mean()['trip_duration'] / 60  # unit in hour
min_2015 = membership_2015.min()['trip_duration'] / 60 # unit in hour  
mid_2015 = membership_2015.quantile(0.5)['trip_duration'] / 60  # unit in hour
#max_2015 = membership_2015.max()['trip_duration'] / 60 # unit in hour ---> too big

member_time_2015 = pd.DataFrame({'Average': mean_2015, '50 percentile': mid_2015, 'Minimum': min_2015})
member_time_2015.plot.bar(figsize=(8,8),fontsize=15)
plt.xlabel('User Duration of Trip Total Count 2015', fontsize=25)

## Trip Count and membership info per city (SF, RC, PA, SJ, MV)

In [ ]:
# filter raw trip data 2015 according to station ID
sf_trip_data_2015 = raw_data_2015[raw_data_2015['start_terminal'].isin(palo_alto_station_id)]
sj_trip_data_2015 = raw_data_2015[raw_data_2015['start_terminal'].isin(san_jose_station_id)]
rc_trip_data_2015 = raw_data_2015[raw_data_2015['start_terminal'].isin(redwood_city_station_id)]
pa_trip_data_2015 = raw_data_2015[raw_data_2015['start_terminal'].isin(palo_alto_station_id)]
mv_trip_data_2015 = raw_data_2015[raw_data_2015['start_terminal'].isin(mountain_view_station_id)]

### San Francisco Trip Count and Membership info

In [ ]:
sf_trip_data_2015 = pd.DataFrame(sf_trip_data_2015, columns=['user_type', 'trip_time'])
sf_trip_data_2015.columns = ['membership', 'trip_duration']  
sf_trip_data_2015.head()

In [ ]:
# SF trip data statistics for both user type
sf_trip_data_2015.describe()

In [ ]:
# membership count (same as trip count) counts in San Francisco
sf_trip_data_membership_2015 = sf_trip_data_2015.groupby('membership')
sf_count_2015 = sf_trip_data_membership_2015.count()['trip_duration']
sf_count_2015.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in San Francisco in Percentage 2015', fontsize=20)

In [ ]:
# SF trip data for each user type
sf_trip_data_membership_2015.describe()

### San Jose Trip Count and Membership info

In [ ]:
sj_trip_data_2015 = pd.DataFrame(sj_trip_data_2015, columns=['user_type', 'trip_time'])
sj_trip_data_2015.columns = ['membership', 'trip_duration']  
sj_trip_data_2015.head()

In [ ]:
# SJ trip data statistics for both user type
sj_trip_data_2015.describe()

In [ ]:
# membership count (same as trip count) counts in San Jose
sj_trip_data_membership_2015 = sj_trip_data_2015.groupby('membership')
sj_count_2015 = sj_trip_data_membership_2015.count()['trip_duration']
sj_count_2015.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in San Jose in Percentage 2015', fontsize=20)

In [ ]:
# SJ trip data for each user type
sj_trip_data_membership_2015.describe()

### Palo Alto Trip Count and Membership info

In [ ]:
pa_trip_data_2015 = pd.DataFrame(pa_trip_data_2015, columns=['user_type', 'trip_time'])
pa_trip_data_2015.columns = ['membership', 'trip_duration']  
pa_trip_data_2015.head()

In [ ]:
# PA trip data statistics for both user type
pa_trip_data_2015.describe()

In [ ]:
# membership count (same as trip count) counts in Palo Alto
pa_trip_data_membership_2015 = pa_trip_data_2015.groupby('membership')
pa_count_2015 = pa_trip_data_membership_2015.count()['trip_duration']
pa_count_2015.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in Palo Alto in Percentage 2015', fontsize=20)

In [ ]:
# PA trip data for each user type
pa_trip_data_membership_2015.describe()

### Redwood CIty Trip Count and Membership info

In [ ]:
rc_trip_data_2015 = pd.DataFrame(rc_trip_data_2015, columns=['user_type', 'trip_time'])
rc_trip_data_2015.columns = ['membership', 'trip_duration']  
rc_trip_data_2015.head()

In [ ]:
# RC trip data statistics for both user type
rc_trip_data_2015.describe()

In [ ]:
# membership count (same as trip count) counts in Redwood City
rc_trip_data_membership_2015 = rc_trip_data_2015.groupby('membership')
rc_count_2015 = rc_trip_data_membership_2015.count()['trip_duration']
rc_count_2015.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in Redwood City in Percentage 2015', fontsize=20)

In [ ]:
# RC trip data for each user type
rc_trip_data_membership_2015.describe()

### Mountain View Trip Count and Membership info

In [ ]:
mv_trip_data_2015 = pd.DataFrame(mv_trip_data_2015, columns=['user_type', 'trip_time'])
mv_trip_data_2015.columns = ['membership', 'trip_duration']  
mv_trip_data_2015.head()

In [ ]:
# MV trip data statistics for both user type
mv_trip_data_2015.describe()

In [ ]:
# membership count (same as trip count) counts in Mountain View
mv_trip_data_membership_2015 = mv_trip_data_2015.groupby('membership')
mv_count_2015 = mv_trip_data_membership_2015.count()['trip_duration']
mv_count_2015.plot.pie(autopct='%.2f', fontsize=25, figsize=(8,8),colors=['g', 'c'])
plt.ylabel('')
plt.xlabel('Trip Count in Mountain View in Percentage 2015', fontsize=20)

In [ ]:
# RC trip data for each user type
mv_trip_data_membership_2015.describe()

In [ ]:
# number of trips for each membership compared with weather (part 3)
# count trip id
# weather for each month and each season